In [1]:
import os
from tqdm import tqdm

def convert_file(fname: str, destination: str, obj_class: str):
    if not os.path.exists(destination):
        os.makedirs(destination)
    with open(fname, 'r') as f:
        for i, line in enumerate(f):
            newline = obj_class + " " + line
            with open(f"{destination}/{i+1:05d}.txt", 'w') as f2:
                f2.write(newline)


def convert_dataset(dataset_dir: str = '.'):
    directories = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]
    for d in tqdm(directories):
        convert_file(f"{d}/{d}.txt", f"{d}/bbox", "0" if "aircraft" in d else "1")

In [2]:
convert_dataset()

100%|██████████| 37/37 [00:00<00:00, 65.86it/s]
